# Chapter 29: Type-Safe API Clients

---

## 29.1 Typing HTTP Responses

Building type-safe API clients starts with accurately modeling HTTP responses. TypeScript enables compile-time verification that API responses match expected shapes, preventing runtime errors from malformed data and providing IntelliSense for response properties.

### 29.1.1 Basic Response Types

Define interfaces that represent the structure of API responses, including data payloads, metadata, and status information.

```typescript
// Basic API response wrapper
interface ApiResponse<T> {
  data: T;
  status: number;
  statusText: string;
  headers: Record<string, string>;
}

// Generic success response
interface SuccessResponse<T> {
  success: true;
  data: T;
  meta?: {
    page?: number;
    pageSize?: number;
    total?: number;
    totalPages?: number;
  };
}

// Error response structure
interface ErrorResponse {
  success: false;
  error: {
    code: string;
    message: string;
    details?: Record<string, string[]>;
  };
}

// Union type for API results
type ApiResult<T> = SuccessResponse<T> | ErrorResponse;

// Usage with fetch
async function fetchUser(id: number): Promise<ApiResult<User>> {
  const response = await fetch(`/api/users/${id}`);
  const result: ApiResult<User> = await response.json();
  
  if (!result.success) {
    throw new Error(result.error.message);
  }
  
  return result;
}

// Strict response validation with branded types
type Validated<T> = T & { __validated: true };

async function fetchValidated<T>(
  url: string,
  validator: (data: unknown) => data is T
): Promise<Validated<T>> {
  const response = await fetch(url);
  const data = await response.json();
  
  if (!validator(data)) {
    throw new Error("Response validation failed");
  }
  
  return data as Validated<T>;
}
```

### 29.1.2 Response Discrimination

Use discriminated unions to handle different response variants with type narrowing.

```typescript
// API response variants
type UserResponse =
  | { type: "user"; data: User }
  | { type: "guest"; data: GuestSession }
  | { type: "error"; error: ApiError };

// Type guard for narrowing
function isUserResponse(response: UserResponse): response is { type: "user"; data: User } {
  return response.type === "user";
}

// Usage with exhaustive checking
async function handleUserResponse(response: UserResponse): Promise<string> {
  switch (response.type) {
    case "user":
      // TypeScript knows response.data is User
      return `Welcome, ${response.data.name}`;
      
    case "guest":
      // TypeScript knows response.data is GuestSession
      return `Guest session: ${response.data.sessionId}`;
      
    case "error":
      // TypeScript knows response.error is ApiError
      throw new Error(response.error.message);
      
    default:
      // Exhaustiveness checking
      const _exhaustive: never = response;
      throw new Error("Unknown response type");
  }
}

// HTTP status code discrimination
type HttpResponse<T> =
  | { status: 200; data: T }
  | { status: 201; data: T; location: string }
  | { status: 400; error: ValidationError }
  | { status: 401; error: AuthenticationError }
  | { status: 403; error: AuthorizationError }
  | { status: 404; error: NotFoundError }
  | { status: 500; error: ServerError };

// Mapped status code handlers
type StatusCodeHandlers<T> = {
  200: (data: T) => void;
  201: (data: T, location: string) => void;
  400: (error: ValidationError) => void;
  401: (error: AuthenticationError) => void;
  403: (error: AuthorizationError) => void;
  404: (error: NotFoundError) => void;
  500: (error: ServerError) => void;
};
```

### 29.1.3 Generic Response Wrappers

Create reusable response types that work across different API endpoints while maintaining type safety.

```typescript
// Paginated response wrapper
interface PaginatedResponse<T> {
  data: T[];
  pagination: {
    currentPage: number;
    pageSize: number;
    totalItems: number;
    totalPages: number;
    hasNextPage: boolean;
    hasPreviousPage: boolean;
  };
  links: {
    self: string;
    first: string;
    last: string;
    next?: string;
    prev?: string;
  };
}

// Cursor-based pagination
interface CursorPaginatedResponse<T> {
  data: T[];
  pageInfo: {
    startCursor: string;
    endCursor: string;
    hasNextPage: boolean;
    hasPreviousPage: boolean;
  };
}

// HATEOAS-style response
interface ResourceResponse<T> {
  data: T;
  links: {
    self: string;
    related?: Record<string, string>;
  };
  embedded?: Record<string, unknown>;
}

// Generic API client response
interface ApiClientResponse<T, E = ApiError> {
  ok: boolean;
  status: number;
  statusText: string;
  data?: T;
  error?: E;
  headers: Headers;
}

// Type-safe response extraction
async function extractData<T>(
  response: ApiClientResponse<T>
): Promise<T> {
  if (!response.ok || response.data === undefined) {
    throw new Error(response.error?.message || "Unknown error");
  }
  return response.data;
}

// Usage
const response: ApiClientResponse<User[]> = await api.get("/users");
const users = await extractData(response); // Type: User[]
```

---

## 29.2 Type-Safe Request Builders

Request builders provide a fluent, type-safe API for constructing HTTP requests with proper typing for parameters, headers, and body payloads.

### 29.2.1 URL Parameter Safety

Type-safe URL construction prevents parameter injection and ensures required parameters are provided.

```typescript
// Path parameter extraction
type PathParams<T extends string> = 
  T extends `${infer _}:${infer Param}/${infer Rest}`
    ? { [K in Param]: string } & PathParams<Rest>
    : T extends `${infer _}:${infer Param}`
      ? { [K in Param]: string }
      : {};

// URL builder with typed parameters
function buildUrl<T extends string>(
  template: T,
  params: PathParams<T>
): string {
  return template.replace(/:([a-zA-Z]+)/g, (_, key) => {
    const value = (params as Record<string, string>)[key];
    if (value === undefined) {
      throw new Error(`Missing parameter: ${key}`);
    }
    return encodeURIComponent(value);
  });
}

// Usage
const userUrl = buildUrl("/users/:userId/posts/:postId", {
  userId: "123",
  postId: "456"
}); // "/users/123/posts/456"

// Query parameter builder
interface QueryParams {
  [key: string]: string | number | boolean | undefined | null;
}

function buildQueryString(params: QueryParams): string {
  const searchParams = new URLSearchParams();
  
  Object.entries(params).forEach(([key, value]) => {
    if (value !== undefined && value !== null) {
      searchParams.append(key, String(value));
    }
  });
  
  const query = searchParams.toString();
  return query ? `?${query}` : "";
}

// Typed query parameters for specific endpoints
interface ListUsersQuery {
  page?: number;
  pageSize?: number;
  sortBy?: "name" | "email" | "createdAt";
  order?: "asc" | "desc";
  search?: string;
  status?: "active" | "inactive";
}

function listUsersQuery(params: ListUsersQuery): string {
  return buildQueryString(params);
}
```

### 29.2.2 Request Body Typing

Ensure request bodies match API expectations with strict typing.

```typescript
// Request body types
interface CreateUserRequest {
  name: string;
  email: string;
  role?: "admin" | "user";
}

interface UpdateUserRequest {
  name?: string;
  email?: string;
  role?: "admin" | "user";
}

// Type-safe request builder
class RequestBuilder<TBody, TResponse> {
  private url: string = "";
  private method: "GET" | "POST" | "PUT" | "PATCH" | "DELETE" = "GET";
  private headers: Headers = new Headers();
  private body?: TBody;

  to(url: string): this {
    this.url = url;
    return this;
  }

  withMethod(method: "GET" | "POST" | "PUT" | "PATCH" | "DELETE"): this {
    this.method = method;
    return this;
  }

  withBody(body: TBody): this {
    this.body = body;
    return this;
  }

  withHeaders(headers: Record<string, string>): this {
    Object.entries(headers).forEach(([key, value]) => {
      this.headers.set(key, value);
    });
    return this;
  }

  async execute(): Promise<TResponse> {
    const response = await fetch(this.url, {
      method: this.method,
      headers: this.headers,
      body: this.body ? JSON.stringify(this.body) : undefined
    });
    
    return response.json() as Promise<TResponse>;
  }
}

// Factory functions for specific endpoints
function createUserRequest(): RequestBuilder<CreateUserRequest, User> {
  return new RequestBuilder<CreateUserRequest, User>()
    .to("/api/users")
    .withMethod("POST")
    .withHeaders({ "Content-Type": "application/json" });
}

// Usage
const newUser = await createUserRequest()
  .withBody({
    name: "John Doe",
    email: "john@example.com",
    role: "user"
  })
  .execute();
```

### 29.2.3 Fluent API Pattern

Create a fluent, chainable API client that maintains type safety throughout the request lifecycle.

```typescript
// Endpoint definitions
interface Endpoints {
  "GET /users": {
    params: void;
    query: { page?: number; pageSize?: number };
    response: PaginatedResponse<User>;
  };
  "GET /users/:id": {
    params: { id: string };
    query: void;
    response: User;
  };
  "POST /users": {
    params: void;
    body: CreateUserRequest;
    response: User;
  };
  "PUT /users/:id": {
    params: { id: string };
    body: UpdateUserRequest;
    response: User;
  };
  "DELETE /users/:id": {
    params: { id: string };
    response: void;
  };
}

// Extract route parameters
type RouteParams<T extends string> = 
  T extends `${infer _}/:${infer Param}/${infer Rest}`
    ? { [K in Param]: string } & RouteParams<`/${Rest}`>
    : T extends `${infer _}/:${infer Param}`
      ? { [K in Param]: string }
      : {};

// Type-safe API client
class TypedApiClient {
  private baseUrl: string;

  constructor(baseUrl: string) {
    this.baseUrl = baseUrl;
  }

  get<T extends keyof Endpoints>(
    route: T extends `GET ${infer R}` ? R : never
  ): Omit<RequestConfig<Endpoints[T]>, "method"> {
    return new RequestConfig<Endpoints[T]>(this.baseUrl, "GET", route);
  }

  post<T extends keyof Endpoints>(
    route: T extends `POST ${infer R}` ? R : never
  ): Omit<RequestConfig<Endpoints[T]>, "method"> {
    return new RequestConfig<Endpoints[T]>(this.baseUrl, "POST", route);
  }

  put<T extends keyof Endpoints>(
    route: T extends `PUT ${infer R}` ? R : never
  ): Omit<RequestConfig<Endpoints[T]>, "method"> {
    return new RequestConfig<Endpoints[T]>(this.baseUrl, "PUT", route);
  }

  delete<T extends keyof Endpoints>(
    route: T extends `DELETE ${infer R}` ? R : never
  ): Omit<RequestConfig<Endpoints[T]>, "method"> {
    return new RequestConfig<Endpoints[T]>(this.baseUrl, "DELETE", route);
  }
}

class RequestConfig<T extends { params: any; query?: any; body?: any; response: any }> {
  private config: {
    url: string;
    method: string;
    params?: T["params"];
    query?: T["query"];
    body?: T["body"];
    headers?: Record<string, string>;
  };

  constructor(baseUrl: string, method: string, route: string) {
    this.config = {
      url: `${baseUrl}${route}`,
      method
    };
  }

  withParams(params: T["params"]): this {
    this.config.params = params;
    // Replace :param in URL
    Object.entries(params as Record<string, string>).forEach(([key, value]) => {
      this.config.url = this.config.url.replace(`:${key}`, encodeURIComponent(value));
    });
    return this;
  }

  withQuery(query: T["query"]): this {
    this.config.query = query;
    if (query && Object.keys(query).length > 0) {
      const queryString = new URLSearchParams(query as Record<string, string>).toString();
      this.config.url += `?${queryString}`;
    }
    return this;
  }

  withBody(body: T["body"]): this {
    this.config.body = body;
    return this;
  }

  withHeaders(headers: Record<string, string>): this {
    this.config.headers = headers;
    return this;
  }

  async execute(): Promise<T["response"]> {
    const response = await fetch(this.config.url, {
      method: this.config.method,
      headers: {
        "Content-Type": "application/json",
        ...this.config.headers
      },
      body: this.config.body ? JSON.stringify(this.config.body) : undefined
    });

    if (!response.ok) {
      throw new Error(`HTTP ${response.status}: ${response.statusText}`);
    }

    return response.json();
  }
}

// Usage
const api = new TypedApiClient("https://api.example.com");

// TypeScript enforces correct parameters
const users = await api
  .get("/users")
  .withQuery({ page: 1, pageSize: 10 })
  .execute();

const user = await api
  .get("/users/:id")
  .withParams({ id: "123" })
  .execute();

const created = await api
  .post("/users")
  .withBody({ name: "John", email: "john@example.com" })
  .execute();
```

---

## 29.3 Handling API Errors

Robust API clients distinguish between expected responses and errors, with typed error handling that provides meaningful context for recovery strategies.

### 29.3.1 Error Classification

Create a hierarchy of error types that mirror API error responses and HTTP status codes.

```typescript
// Base API error
class ApiError extends Error {
  constructor(
    message: string,
    public statusCode: number,
    public code: string,
    public details?: Record<string, string[]>
  ) {
    super(message);
    this.name = "ApiError";
  }
}

// Specific error types
class ValidationError extends ApiError {
  constructor(details: Record<string, string[]>) {
    super("Validation failed", 400, "VALIDATION_ERROR", details);
    this.name = "ValidationError";
  }
}

class AuthenticationError extends ApiError {
  constructor(message: string = "Authentication required") {
    super(message, 401, "AUTHENTICATION_ERROR");
    this.name = "AuthenticationError";
  }
}

class AuthorizationError extends ApiError {
  constructor(message: string = "Permission denied") {
    super(message, 403, "AUTHORIZATION_ERROR");
    this.name = "AuthorizationError";
  }
}

class NotFoundError extends ApiError {
  constructor(resource: string) {
    super(`${resource} not found`, 404, "NOT_FOUND");
    this.name = "NotFoundError";
  }
}

class ConflictError extends ApiError {
  constructor(message: string) {
    super(message, 409, "CONFLICT");
    this.name = "ConflictError";
  }
}

class RateLimitError extends ApiError {
  public retryAfter: number;

  constructor(retryAfter: number) {
    super("Rate limit exceeded", 429, "RATE_LIMIT");
    this.retryAfter = retryAfter;
    this.name = "RateLimitError";
  }
}

class ServerError extends ApiError {
  constructor(message: string = "Internal server error") {
    super(message, 500, "SERVER_ERROR");
    this.name = "ServerError";
  }
}

// Error factory based on status code
function createError(response: Response, data?: any): ApiError {
  switch (response.status) {
    case 400:
      return new ValidationError(data?.errors || {});
    case 401:
      return new AuthenticationError(data?.message);
    case 403:
      return new AuthorizationError(data?.message);
    case 404:
      return new NotFoundError(data?.resource || "Resource");
    case 409:
      return new ConflictError(data?.message);
    case 429:
      return new RateLimitError(parseInt(response.headers.get("Retry-After") || "60"));
    case 500:
    case 502:
    case 503:
    case 504:
      return new ServerError(data?.message);
    default:
      return new ApiError(
        data?.message || "Unknown error",
        response.status,
        "UNKNOWN_ERROR"
      );
  }
}
```

### 29.3.2 Error Handling Strategies

Implement different strategies for handling errors based on context and recoverability.

```typescript
// Result type for explicit error handling
type Result<T, E = ApiError> = 
  | { success: true; data: T }
  | { success: false; error: E };

// Safe wrapper that never throws
async function safeApiCall<T>(
  operation: () => Promise<T>
): Promise<Result<T>> {
  try {
    const data = await operation();
    return { success: true, data };
  } catch (error) {
    if (error instanceof ApiError) {
      return { success: false, error };
    }
    return {
      success: false,
      error: new ApiError(
        error instanceof Error ? error.message : "Unknown error",
        0,
        "UNKNOWN"
      )
    };
  }
}

// Retry strategy with exponential backoff
interface RetryConfig {
  maxRetries: number;
  baseDelay: number;
  maxDelay: number;
  retryableStatuses: number[];
}

const defaultRetryConfig: RetryConfig = {
  maxRetries: 3,
  baseDelay: 1000,
  maxDelay: 10000,
  retryableStatuses: [408, 429, 500, 502, 503, 504]
};

async function withRetry<T>(
  operation: () => Promise<T>,
  config: Partial<RetryConfig> = {}
): Promise<T> {
  const fullConfig = { ...defaultRetryConfig, ...config };
  let lastError: Error | undefined;

  for (let attempt = 0; attempt <= fullConfig.maxRetries; attempt++) {
    try {
      return await operation();
    } catch (error) {
      lastError = error as Error;
      
      if (error instanceof ApiError) {
        // Don't retry client errors (4xx except specific ones)
        if (error.statusCode >= 400 && error.statusCode < 500) {
          if (!fullConfig.retryableStatuses.includes(error.statusCode)) {
            throw error;
          }
        }
      }

      if (attempt < fullConfig.maxRetries) {
        const delay = Math.min(
          fullConfig.baseDelay * Math.pow(2, attempt),
          fullConfig.maxDelay
        );
        await new Promise(resolve => setTimeout(resolve, delay));
      }
    }
  }

  throw lastError;
}

// Circuit breaker pattern
class CircuitBreaker {
  private failures = 0;
  private lastFailureTime?: number;
  private state: "CLOSED" | "OPEN" | "HALF_OPEN" = "CLOSED";

  constructor(
    private threshold = 5,
    private timeout = 60000
  ) {}

  async execute<T>(operation: () => Promise<T>): Promise<T> {
    if (this.state === "OPEN") {
      if (Date.now() - (this.lastFailureTime || 0) > this.timeout) {
        this.state = "HALF_OPEN";
      } else {
        throw new Error("Circuit breaker is OPEN");
      }
    }

    try {
      const result = await operation();
      this.onSuccess();
      return result;
    } catch (error) {
      this.onFailure();
      throw error;
    }
  }

  private onSuccess(): void {
    this.failures = 0;
    this.state = "CLOSED";
  }

  private onFailure(): void {
    this.failures++;
    this.lastFailureTime = Date.now();
    
    if (this.failures >= this.threshold) {
      this.state = "OPEN";
    }
  }
}
```

### 29.3.3 Error Boundaries

Implement error boundaries that isolate failures and provide fallback behavior.

```typescript
// API client with error boundary
class ResilientApiClient {
  private circuitBreaker = new CircuitBreaker();
  private cache = new Map<string, unknown>();

  async fetchWithFallback<T>(
    key: string,
    operation: () => Promise<T>,
    fallback?: T
  ): Promise<T> {
    try {
      // Try cache first
      if (this.cache.has(key)) {
        return this.cache.get(key) as T;
      }

      // Execute with circuit breaker
      const result = await this.circuitBreaker.execute(operation);
      
      // Cache successful result
      this.cache.set(key, result);
      return result;
    } catch (error) {
      // Return fallback if available
      if (fallback !== undefined) {
        console.warn(`Using fallback for ${key}:`, error);
        return fallback;
      }
      throw error;
    }
  }

  async fetchWithTimeout<T>(
    operation: () => Promise<T>,
    timeoutMs: number
  ): Promise<T> {
    return Promise.race([
      operation(),
      new Promise<T>((_, reject) =>
        setTimeout(() => reject(new Error("Request timeout")), timeoutMs)
      )
    ]);
  }
}

// React-style error boundary (for UI components)
interface ErrorBoundaryState<T> {
  data?: T;
  error?: ApiError;
  loading: boolean;
}

class ApiResource<T> {
  private state: ErrorBoundaryState<T> = { loading: false };
  private listeners: Set<(state: ErrorBoundaryState<T>) => void> = new Set();

  async fetch(fetcher: () => Promise<T>): Promise<void> {
    this.setState({ loading: true, error: undefined });
    
    try {
      const data = await fetcher();
      this.setState({ data, loading: false });
    } catch (error) {
      this.setState({
        error: error instanceof ApiError ? error : new ApiError(String(error), 0, "UNKNOWN"),
        loading: false
      });
    }
  }

  private setState(newState: Partial<ErrorBoundaryState<T>>): void {
    this.state = { ...this.state, ...newState };
    this.listeners.forEach(listener => listener(this.state));
  }

  subscribe(listener: (state: ErrorBoundaryState<T>) => void): () => void {
    this.listeners.add(listener);
    listener(this.state);
    return () => this.listeners.delete(listener);
  }
}
```

---

## 29.4 API Client Patterns

Architectural patterns for organizing API clients that scale with application complexity while maintaining type safety.

### 29.4.1 Repository Pattern

Encapsulate data access logic in repository classes that abstract the underlying API implementation.

```typescript
// Base repository interface
interface Repository<T, CreateDTO, UpdateDTO> {
  findAll(query?: QueryOptions): Promise<PaginatedResponse<T>>;
  findById(id: string): Promise<T | null>;
  create(data: CreateDTO): Promise<T>;
  update(id: string, data: UpdateDTO): Promise<T>;
  delete(id: string): Promise<void>;
}

// User repository implementation
interface UserCreateDTO {
  name: string;
  email: string;
  role: "admin" | "user";
}

interface UserUpdateDTO {
  name?: string;
  email?: string;
  role?: "admin" | "user";
}

class UserRepository implements Repository<User, UserCreateDTO, UserUpdateDTO> {
  private basePath = "/api/users";

  constructor(private client: ApiClient) {}

  async findAll(options?: QueryOptions): Promise<PaginatedResponse<User>> {
    const query = this.buildQuery(options);
    return this.client.get<PaginatedResponse<User>>(this.basePath, query);
  }

  async findById(id: string): Promise<User | null> {
    try {
      return await this.client.get<User>(`${this.basePath}/${id}`);
    } catch (error) {
      if (error instanceof NotFoundError) {
        return null;
      }
      throw error;
    }
  }

  async create(data: UserCreateDTO): Promise<User> {
    return this.client.post<User>(this.basePath, data);
  }

  async update(id: string, data: UserUpdateDTO): Promise<User> {
    return this.client.patch<User>(`${this.basePath}/${id}`, data);
  }

  async delete(id: string): Promise<void> {
    await this.client.delete(`${this.basePath}/${id}`);
  }

  // Custom repository methods
  async findByEmail(email: string): Promise<User | null> {
    const result = await this.findAll({ filters: { email } });
    return result.data[0] || null;
  }

  async activate(id: string): Promise<User> {
    return this.client.post<User>(`${this.basePath}/${id}/activate`);
  }

  private buildQuery(options?: QueryOptions): Record<string, string> {
    // Implementation
    return {};
  }
}

// Usage
const userRepo = new UserRepository(apiClient);
const user = await userRepo.findById("123");
if (user) {
  await userRepo.update(user.id, { name: "New Name" });
}
```

### 29.4.2 Service Layer Pattern

Implement a service layer that orchestrates repositories and encapsulates business logic.

```typescript
// Domain service
interface AuthService {
  login(credentials: LoginCredentials): Promise<AuthResult>;
  logout(): Promise<void>;
  refreshToken(): Promise<string>;
  getCurrentUser(): Promise<User | null>;
  isAuthenticated(): boolean;
}

interface LoginCredentials {
  email: string;
  password: string;
  rememberMe?: boolean;
}

interface AuthResult {
  user: User;
  accessToken: string;
  refreshToken: string;
  expiresAt: Date;
}

class ApiAuthService implements AuthService {
  private tokenStorage: TokenStorage;
  private currentUser?: User;

  constructor(
    private apiClient: ApiClient,
    private userRepository: UserRepository
  ) {
    this.tokenStorage = new TokenStorage();
  }

  async login(credentials: LoginCredentials): Promise<AuthResult> {
    const response = await this.apiClient.post<{
      user: User;
      tokens: { access: string; refresh: string; expiresIn: number };
    }>("/api/auth/login", credentials);

    const { user, tokens } = response;
    const expiresAt = new Date(Date.now() + tokens.expiresIn * 1000);

    this.tokenStorage.setTokens({
      access: tokens.access,
      refresh: tokens.refresh,
      expiresAt
    });

    this.currentUser = user;
    this.setupTokenRefresh(expiresAt);

    return {
      user,
      accessToken: tokens.access,
      refreshToken: tokens.refresh,
      expiresAt
    };
  }

  async logout(): Promise<void> {
    try {
      await this.apiClient.post("/api/auth/logout");
    } finally {
      this.tokenStorage.clear();
      this.currentUser = undefined;
    }
  }

  async refreshToken(): Promise<string> {
    const refreshToken = this.tokenStorage.getRefreshToken();
    if (!refreshToken) {
      throw new AuthenticationError("No refresh token available");
    }

    const response = await this.apiClient.post<{
      accessToken: string;
      expiresIn: number;
    }>("/api/auth/refresh", { refreshToken });

    const expiresAt = new Date(Date.now() + response.expiresIn * 1000);
    this.tokenStorage.setAccessToken(response.accessToken, expiresAt);
    this.setupTokenRefresh(expiresAt);

    return response.accessToken;
  }

  async getCurrentUser(): Promise<User | null> {
    if (this.currentUser) {
      return this.currentUser;
    }

    try {
      const user = await this.apiClient.get<User>("/api/auth/me");
      this.currentUser = user;
      return user;
    } catch (error) {
      if (error instanceof AuthenticationError) {
        return null;
      }
      throw error;
    }
  }

  isAuthenticated(): boolean {
    return this.tokenStorage.hasValidToken();
  }

  private setupTokenRefresh(expiresAt: Date): void {
    const refreshTime = expiresAt.getTime() - Date.now() - 60000; // 1 minute before expiry
    setTimeout(() => this.refreshToken(), refreshTime);
  }
}

// Token storage implementation
class TokenStorage {
  private readonly ACCESS_KEY = "access_token";
  private readonly REFRESH_KEY = "refresh_token";
  private readonly EXPIRES_KEY = "token_expires";

  setTokens(tokens: { access: string; refresh: string; expiresAt: Date }): void {
    localStorage.setItem(this.ACCESS_KEY, tokens.access);
    localStorage.setItem(this.REFRESH_KEY, tokens.refresh);
    localStorage.setItem(this.EXPIRES_KEY, tokens.expiresAt.toISOString());
  }

  setAccessToken(token: string, expiresAt: Date): void {
    localStorage.setItem(this.ACCESS_KEY, token);
    localStorage.setItem(this.EXPIRES_KEY, expiresAt.toISOString());
  }

  getAccessToken(): string | null {
    return localStorage.getItem(this.ACCESS_KEY);
  }

  getRefreshToken(): string | null {
    return localStorage.getItem(this.REFRESH_KEY);
  }

  hasValidToken(): boolean {
    const expires = localStorage.getItem(this.EXPIRES_KEY);
    if (!expires) return false;
    return new Date(expires) > new Date();
  }

  clear(): void {
    localStorage.removeItem(this.ACCESS_KEY);
    localStorage.removeItem(this.REFRESH_KEY);
    localStorage.removeItem(this.EXPIRES_KEY);
  }
}
```

### 29.4.3 API Client Factory

Create configurable API client instances with interceptors, middleware, and environment-specific settings.

```typescript
// API client configuration
interface ApiClientConfig {
  baseURL: string;
  timeout: number;
  retries: number;
  headers?: Record<string, string>;
}

// Request/Response interceptors
interface Interceptor {
  request?: (config: RequestInit) => RequestInit | Promise<RequestInit>;
  response?: (response: Response) => Response | Promise<Response>;
  error?: (error: ApiError) => ApiError | Promise<ApiError>;
}

class ApiClientFactory {
  private interceptors: Interceptor[] = [];
  private config: ApiClientConfig;

  constructor(config: Partial<ApiClientConfig> = {}) {
    this.config = {
      baseURL: "",
      timeout: 10000,
      retries: 3,
      ...config
    };
  }

  setBaseURL(url: string): this {
    this.config.baseURL = url;
    return this;
  }

  setTimeout(timeout: number): this {
    this.config.timeout = timeout;
    return this;
  }

  addInterceptor(interceptor: Interceptor): this {
    this.interceptors.push(interceptor);
    return this;
  }

  addAuthInterceptor(getToken: () => string | null): this {
    this.addInterceptor({
      request: (config) => {
        const token = getToken();
        if (token) {
          config.headers = {
            ...config.headers,
            Authorization: `Bearer ${token}`
          };
        }
        return config;
      }
    });
    return this;
  }

  addLoggingInterceptor(): this {
    this.addInterceptor({
      request: (config) => {
        console.log(`[API] ${config.method} ${config.url}`);
        return config;
      },
      response: (response) => {
        console.log(`[API] Response: ${response.status}`);
        return response;
      },
      error: (error) => {
        console.error(`[API] Error:`, error);
        return error;
      }
    });
    return this;
  }

  create(): ApiClient {
    return new ApiClient(this.config, this.interceptors);
  }
}

// Usage
const apiClient = new ApiClientFactory()
  .setBaseURL("https://api.example.com")
  .setTimeout(5000)
  .addAuthInterceptor(() => localStorage.getItem("token"))
  .addLoggingInterceptor()
  .create();
```

---

## 29.5 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored building type-safe API clients:

**Key Takeaways:**

1. **Response Typing**:
   - Use generic interfaces for flexible response wrappers
   - Implement discriminated unions for variant responses
   - Validate responses at runtime when necessary

2. **Request Builders**:
   - Type-safe URL construction with path parameters
   - Fluent API patterns for request configuration
   - Generic constraints for endpoint definitions

3. **Error Handling**:
   - Hierarchical error classes for different failure modes
   - Retry strategies with exponential backoff
   - Circuit breakers for fault tolerance
   - Graceful degradation with fallbacks

4. **Architectural Patterns**:
   - Repository pattern for data access abstraction
   - Service layer for business logic encapsulation
   - Factory pattern for configurable client instances
   - Interceptors for cross-cutting concerns

### Practical Exercises

**Exercise 1: Response Typing**

Create typed API responses:

```typescript
// Create response types for:
// 1. A generic ApiResponse<T> with data, status, and error
// 2. A PaginatedResponse<T> with items and pagination metadata
// 3. A discriminated union for different response types (success, error, loading)
// 4. Type guards for narrowing response types

// Test with a User API that returns:
// - Single user by ID
// - List of users with pagination
// - Validation errors for invalid input
```

**Exercise 2: Request Builder**

Build a fluent API client:

```typescript
// Create a RequestBuilder that:
// 1. Supports method chaining (url().method().body().execute())
// 2. Types path parameters using template literal types
// 3. Validates request bodies against interfaces
// 4. Returns typed responses

// Implement endpoints:
// GET /users (paginated)
// GET /users/:id (single)
// POST /users (create)
// PUT /users/:id (update)
// DELETE /users/:id (delete)

// Ensure TypeScript catches:
// - Missing path parameters
// - Incorrect body shapes
// - Wrong HTTP methods
```

**Exercise 3: Error Handling**

Implement error strategies:

```typescript
// Create:
// 1. Custom error classes for different HTTP status codes
// 2. A retry mechanism with exponential backoff
// 3. A circuit breaker that opens after 5 failures
// 4. A safe wrapper that returns Result<T, Error> instead of throwing

// Test with:
// - Simulated network failures
// - 404 and 500 responses
// - Timeout scenarios
// - Recovery after failures
```

**Exercise 4: Repository Pattern**

Build a complete repository:

```typescript
// Create a Repository<T> base class with:
// - findAll, findById, create, update, delete methods
// - Caching layer with TTL
// - Optimistic updates

// Implement concrete ProductRepository with:
// - Custom methods (findByCategory, updateInventory)
// - Search functionality
// - Image upload handling

// Ensure type safety throughout
```

### Additional Resources

- **TypeScript HTTP Clients**: https://github.com/axios/axios (with @types/axios)
- **Fetch API Types**: https://developer.mozilla.org/en-US/docs/Web/API/Fetch_API
- **Repository Pattern**: https://martinfowler.com/eaaCatalog/repository.html
- **Circuit Breaker Pattern**: https://martinfowler.com/bliki/CircuitBreaker.html

---

## Coming Up Next: Chapter 30 - Dependency Injection in TypeScript

In the next chapter, we will explore **Dependency Injection**, a fundamental pattern for building testable, maintainable applications:

- **30.1 Understanding Dependency Injection** - Core concepts and benefits
- **30.2 DI Container Patterns** - Implementing inversion of control
- **30.3 Type-Safe Injection Tokens** - Symbols, types, and decorators
- **30.4 Framework Integration** - Using InversifyJS, TSyringe, and NestJS
- **30.5 Manual DI vs Frameworks** - When to use each approach

Dependency injection enables loose coupling between components, making code easier to test, maintain, and evolve. TypeScript's type system makes DI containers particularly powerful, ensuring that dependencies are resolved correctly at compile time.